# Multi-Agent Workflows with NuGen
<img src="https://nugen.in/logo.png" alt="Nugen Logo" width="180"/>

**Designing Collaborative AI Systems using NuGen Agents**

Domain-aligned foundational models at industry-leading speeds with zero data retention.  
Learn more at: https://docs.nugen.in/introduction

## Introduction

Most AI applications rely on a single agent handling every task — planning, reasoning,
research, validation, and response generation. This approach often breaks down when
tasks become complex, multi-step, or high-stakes.

In this cookbook, we explore **Agent-Oriented Systems** using NuGen Agents — where
multiple specialized agents collaborate like a real-world team.

You will learn how to:
- Design specialized AI agents with clear responsibilities
- Orchestrate agents into a workflow
- Improve reliability, reasoning, and output quality
- Debug and troubleshoot agent behavior

## Problem Statement

Single-agent systems often suffer from:
- Cognitive overload
- Hallucinations
- Poor reasoning on multi-step tasks
- Lack of verification

We need a structured approach where **each agent does one thing well**.

## Solution: Multi-Agent Workflow Architecture

We will build a **4-Agent system**:

1. **Planner Agent** – Breaks the task into steps
2. **Research Agent** – Gathers structured information
3. **Validator Agent** – Checks factual correctness & gaps
4. **Writer Agent** – Produces the final response

Each agent is:
- Independently configurable
- Powered by NuGen models
- Optimized using instructions + demonstrations

## Use Case: Policy Research Assistant

**User Input:**
> "Analyze India's EV policy and summarize opportunities for startups."

**Expected Outcome:**
A structured, validated, high-quality report created through agent collaboration.

## Prerequisites

Before proceeding, ensure you have:

- A valid **NuGen API Key**
- Python 3.9+
- Basic understanding of REST APIs

## Environment Setup

In [1]:
!pip install --quiet requests python-dotenv

In [2]:
import os
import requests
import json
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("NUGEN_API_KEY")
BASE_URL = "https://api.nugen.in/agents"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

## Designing Specialized Agents

Each agent should:
- Have **one clear responsibility**
- Use **custom instructions**
- Use **task-appropriate temperature**
- Include **demonstrations** when needed

## Planner Agent

**Role:** Convert user intent into a step-by-step execution plan  
**Temperature:** Low (deterministic reasoning)

In [4]:
def create_planner_agent():
    payload = {
        "agent_name": "Planner Agent",
        "agent_description": "Breaks tasks into structured steps",
        "model": "nugen-flash-instruct",
        "temperature": 0.2,
        "instructions": (
            "You are a planning agent. Your job is to break the user request "
            "into clear, logical steps. Do not answer the question directly."
        )
    }
    return requests.post(f"{BASE_URL}/create/", json=payload, headers=headers).json()

planner_agent = create_planner_agent()
planner_id = "planner_agent"

## Research Agent

**Role:** Gather domain knowledge based on the plan  
**Temperature:** Medium (balanced reasoning)

In [6]:
def create_research_agent():
    payload = {
        "agent_name": "Research Agent",
        "agent_description": "Performs factual research",
        "model": "nugen-flash-instruct",
        "temperature": 0.5,
        "instructions": (
            "You are a research agent. Provide factual, structured information. "
            "Avoid speculation."
        )
    }
    return requests.post(f"{BASE_URL}/create/", json=payload, headers=headers).json()

research_agent = create_research_agent()
research_id = "research_agent"

## Validator Agent

**Role:** Detect hallucinations and missing logic  
**Temperature:** Very Low

In [7]:
def create_validator_agent():
    payload = {
        "agent_name": "Validator Agent",
        "agent_description": "Validates factual correctness",
        "model": "nugen-flash-instruct",
        "temperature": 0.1,
        "instructions": (
            "You are a validation agent. Identify factual errors, unsupported claims, "
            "and missing context."
        )
    }
    return requests.post(f"{BASE_URL}/create/", json=payload, headers=headers).json()

validator_agent = create_validator_agent()
validator_id = "validator_agent"

## Writer Agent

**Role:** Generate final user-facing response  
**Temperature:** Medium-High

In [8]:
def create_writer_agent():
    payload = {
        "agent_name": "Writer Agent",
        "agent_description": "Produces final polished response",
        "model": "nugen-flash-instruct",
        "temperature": 0.7,
        "instructions": (
            "You are a professional writer. Create clear, concise, and structured responses."
        )
    }
    return requests.post(f"{BASE_URL}/create/", json=payload, headers=headers).json()

writer_agent = create_writer_agent()
writer_id = "writer_agent"

## Agent Orchestration Logic

The workflow follows this sequence:

User Input  
→ Planner Agent  
→ Research Agent  
→ Validator Agent  
→ Writer Agent

In [9]:
def run_agent(agent_id, message):
    payload = {"message": message}
    response = requests.post(
        f"{BASE_URL}/{agent_id}/run/",
        json=payload,
        headers=headers
    )
    return response.json()["response"]

In [10]:
def run_agent(agent_id, prompt):
    response = client.run(
        agent_id=agent_id,
        input=prompt
    )

    data = response.json()

    if "message" in data:
        return data["message"]

    if "detail" in data:
        raise RuntimeError(f"Model error: {data['detail']}")

    raise RuntimeError(f"Unknown response format: {data}")

In [11]:
!pip install nugen

In [12]:
!pip install requests

In [13]:
import requests
import os

NUGEN_API_KEY = os.getenv("NUGEN_API_KEY")

BASE_URL = "https://api.nugen.in/api/v3/agents/run-agents/"

In [14]:
import requests
import os

BASE_URL = "https://api.nugen.in/api/v3/agents/run-agents/"
NUGEN_API_KEY = os.getenv("NUGEN_API_KEY")

def run_agent(agent_name, prompt):
    url = f"{BASE_URL}{agent_name}/run/"

    headers = {
        "Authorization": f"Bearer {NUGEN_API_KEY}",
        "Content-Type": "application/json"
    }

    payload = {
        "message": prompt
    }

    response = requests.post(url, headers=headers, json=payload)

    print("STATUS:", response.status_code)
    print("RAW TEXT:", response.text)

    try:
        data = response.json()
        print("JSON:", data)
    except Exception:
        raise RuntimeError("Response is not JSON")

    return data

In [15]:
run_agent("planner_agent", "Say hello in one sentence")

STATUS: 429
RAW TEXT: {"detail":"Quota limit reached. Please add credits to your account to continue using the API."}
JSON: {'detail': 'Quota limit reached. Please add credits to your account to continue using the API.'}


{'detail': 'Quota limit reached. Please add credits to your account to continue using the API.'}

### Example Output

The final response is:
- Structured
- Fact-checked
- Clear
- Ready for real-world use

## Troubleshooting Guide

**Planner too vague**
- Reduce temperature
- Add step-format constraints

**Research hallucinations**
- Add validation agent
- Lower temperature

**Overly verbose responses**
- Tighten writer instructions

## Best Practices for Multi-Agent Systems

- One responsibility per agent
- Deterministic agents for reasoning
- Creative agents only at final stage
- Always validate before publishing

## Conclusion

This cookbook demonstrated how NuGen Agents can be combined into
powerful multi-agent workflows that outperform single-agent systems.

You now have a reusable blueprint for:
- Research assistants
- Policy analysis tools
- Enterprise AI workflows
- Reliable agentic systems